### **COMPORTEMENT DYNAMIQUE D'UN RÉSEAU RECURRENT**

#### **Données**

In [39]:
from numpy import array,random,triu

In [19]:
matrice_poids = [[0, -1, 1, 1],
                 [-1, 0, 1, -1],
                 [1, 1, 0, 1],
                 [1, -1, 1, 0]]
states = [[0,0,0,0], [0,0,0,1], [0,0,1,0], [0,0,1,1],
           [0,1,0,0], [0,1,0,1], [0,1,1,0], [0,1,1,1],
           [1,0,0,0], [1,0,0,1], [1,0,1,0], [1,0,1,1],
           [1,1,0,0], [1,1,0,1], [1,1,1,0], [1,1,1,1]]

f_activation = lambda x: 1 if x > 0 else 0

# on part de la config "0" :

initial_state = [0, 0, 0, 0]

# testons la cellule "a" aka "cellule 0" :

cellule=0


In [37]:
# fonction qui passe d'un entier au vecteur contenant la sequence de bits correspondante
def int_vers_binvec (x,nb_bits) :
  tmp = f'{bin(x):>{nb_bits+2}}'.replace("0b", "").replace(" ", "0")
  config = array ([0 for _ in range(nb_bits)])
  for cellule in range(nb_bits) :
   config[cellule]=int(tmp[cellule])
  return config

# fonction qui passe d'un vecteur contenant une sequence de bits a l'entier correspondant
def binvec_vers_int (config): 
  x = "0b"
  for i in range(len(config)):
    x += str(config[i])
  return int(x, 2)
  

In [38]:
int_vers_binvec(1023,10)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

#### **La classe reseau recurrent**

In [22]:
# La classe reseau recurrent 
class ReseauRecurrent:
    def __init__(self,matrice) -> None:
        self.matrice = matrice
    
    # Une fonction qui retourne l'état résultant d'un influ synchrone en partant d'un état
    def influ_synchrone(self,state):
        # On initialise une liste [0,0,....] pour le nouvel état  
        new_state = [0]*len(state)
        #On calcule les nouvelles valeurs pour cet état
        for i in range (len(state)):
           s = 0
           for j in range(len(state)):
              s += self.matrice[j][i]*state[j]
           new_state[i]=f_activation(s)
        
        return new_state

    # Une fonction qui retourne l'état résultant d'un influ asynchrone en partant d'un état

    def influ_asynchrone(self,state):
        for i in range (len(state)):
          s = 0
          for j in range(len(state)):
              s+= self.matrice[j][i]*state[j]
          state[i] = f_activation(s)
        return state
    
    # Une fonction qui prend en paramètre l'état initial du réseau et affiche son évolution
    def evolution(self,state,synchrone = True):
        # une liste qui contiendra l'état précédent,actuel et suivant
        p_a_s =[[],[],state[:]]
        new_state = state[:]
        print(f"=============INITIAL STATE {state} =============\n")
        print(f"{state}",end=" => ")
        while True:
            # Si synchrone est True, alors les influs sont synchrones
            if(synchrone):
                new_state=self.influ_synchrone(new_state)
            # Sinon, asynchone
            else:
                new_state=self.influ_asynchrone(new_state)
            p_a_s.append(new_state)
            p_a_s.pop(0)
            # Etat stable , l'état actuelle et le suivant sont les mêmes 
            if(p_a_s[1]==p_a_s[2]):
                print(f"Etat stable {p_a_s[1]} \n")
                break
            # Y a un cycle si l'état suivant notre état actuel est le même que son état précédent
            elif(p_a_s[0]==p_a_s[2]):
                print(f"Nous avons un cycle {p_a_s[0]} => {p_a_s[1]} => {p_a_s[2]}\n")
                break
            else:
                print(new_state, end=" => ")






#### **Tests**

In [23]:
# Test sur quelques états 
reseau_recurent = ReseauRecurrent(matrice=matrice_poids)
print(reseau_recurent.influ_synchrone([1,1,0,1]))
print(reseau_recurent.influ_synchrone([0,0,1,0]))
print(reseau_recurent.influ_asynchrone([1,1,0,1]))

[0, 0, 1, 0]
[1, 1, 0, 1]
[0, 0, 1, 1]


#### **EVOLUTION DU RESEAU EN PARTANT D'UN ETAT INITIAL DONNÉ**

##### Evolution à partir d'un état initial

In [24]:
reseau_recurent.evolution(initial_state)

=============INITIAL STATE [1, 0, 1, 0] =============

[1, 0, 1, 0] => [1, 0, 1, 1] => Etat stable [1, 0, 1, 1] 



##### Evolutions en partant de tous les etats initials possibles

In [25]:
for state in states:
    reseau_recurent.evolution(state,synchrone=True)

=============INITIAL STATE [0, 0, 0, 0] =============

[0, 0, 0, 0] => Etat stable [0, 0, 0, 0] 

=============INITIAL STATE [0, 0, 0, 1] =============

[0, 0, 0, 1] => [1, 0, 1, 0] => [1, 0, 1, 1] => Etat stable [1, 0, 1, 1] 

=============INITIAL STATE [0, 0, 1, 0] =============

[0, 0, 1, 0] => [1, 1, 0, 1] => Nous avons un cycle [0, 0, 1, 0] => [1, 1, 0, 1] => [0, 0, 1, 0]

=============INITIAL STATE [0, 0, 1, 1] =============

[0, 0, 1, 1] => [1, 0, 1, 1] => Etat stable [1, 0, 1, 1] 

=============INITIAL STATE [0, 1, 0, 0] =============

[0, 1, 0, 0] => [0, 0, 1, 0] => [1, 1, 0, 1] => Nous avons un cycle [0, 0, 1, 0] => [1, 1, 0, 1] => [0, 0, 1, 0]

=============INITIAL STATE [0, 1, 0, 1] =============

[0, 1, 0, 1] => [0, 0, 1, 0] => [1, 1, 0, 1] => Nous avons un cycle [0, 0, 1, 0] => [1, 1, 0, 1] => [0, 0, 1, 0]

=============INITIAL STATE [0, 1, 1, 0] =============

[0, 1, 1, 0] => Etat stable [0, 1, 1, 0] 

=============INITIAL STATE [0, 1, 1, 1] =============

[0, 1, 1, 1] =

In [26]:
for state in states:
        reseau_recurent.evolution(state,synchrone=False)

=============INITIAL STATE [0, 0, 0, 0] =============

[0, 0, 0, 0] => Etat stable [0, 0, 0, 0] 

=============INITIAL STATE [0, 0, 0, 1] =============

[0, 0, 0, 1] => [1, 0, 1, 1] => Etat stable [1, 0, 1, 1] 

=============INITIAL STATE [0, 0, 1, 0] =============

[0, 0, 1, 0] => [1, 0, 1, 1] => Etat stable [1, 0, 1, 1] 

=============INITIAL STATE [0, 0, 1, 1] =============

[0, 0, 1, 1] => [1, 0, 1, 1] => Etat stable [1, 0, 1, 1] 

=============INITIAL STATE [0, 1, 0, 0] =============

[0, 1, 0, 0] => [0, 0, 0, 0] => Etat stable [0, 0, 0, 0] 

=============INITIAL STATE [0, 1, 0, 1] =============

[0, 1, 0, 1] => [0, 0, 1, 1] => Etat stable [1, 0, 1, 1] 

=============INITIAL STATE [0, 1, 1, 0] =============

[0, 1, 1, 0] => Etat stable [0, 1, 1, 0] 

=============INITIAL STATE [0, 1, 1, 1] =============

[0, 1, 1, 1] => [1, 0, 1, 1] => Etat stable [1, 0, 1, 1] 

=============INITIAL STATE [1, 0, 0, 0] =============

[1, 0, 0, 0] => [0, 0, 0, 0] => Etat stable [0, 0, 0, 0] 

======

#### **ANALYSE DES ÉTATS STABLES ET CYCLES SUR UN PLUS GRAND RÉSEAU**

In [29]:
# La matrice de poids (10*10) symétrique et aléatoire 
# On crée une matrice 10*10 rempli de 1 et -1 aléatoirement 
matrice10 = random.choice([-1,1],size=(10,10))
# On garde son triangle supérieur
matrice10 = triu(matrice10,1)
# On lui ajoute sa transposée 
matrice10 += matrice10.T
matrice10
# Notre réseau récurrent de 10 cellules 
reseau_reccurent10 = ReseauRecurrent(matrice=matrice10)
reseau_reccurent10.evolution([random.choice([0,1]) for _ in range(10)])

=============INITIAL STATE [1, 1, 0, 1, 0, 0, 0, 1, 1, 1] =============

[1, 1, 0, 1, 0, 0, 0, 1, 1, 1] => [1, 1, 0, 1, 0, 0, 1, 0, 0, 0] => [1, 1, 0, 1, 0, 0, 1, 0, 1, 0] => [1, 1, 0, 0, 0, 0, 1, 0, 1, 0] => Etat stable [1, 1, 0, 0, 0, 0, 1, 0, 1, 0] 

